In [235]:
def stock_value(stock):
    # Import Libraries
    import requests
    import bs4
    from bs4 import BeautifulSoup as soup
    import re
    import math

    # URL codes
    stock_general = 'https://finance.yahoo.com/quote/'
    stock_bs_1 = 'https://finance.yahoo.com/quote/'
    stock_bs_2 = '/balance-sheet?p='
    stock_ks_1 = 'https://finance.yahoo.com/quote/'
    stock_ks_2 = '/key-statistics/'

    # HTML
    url_stock = stock_general + stock + '/'
    url_stock_bs = stock_bs_1 + stock + stock_bs_2 + stock
    url_stock_ks = stock_ks_1 + stock + stock_ks_2

    # Create soup from the general stock info
    html_gen = requests.get(url_stock)
    soup_gen = bs4.BeautifulSoup(html_gen.content, 'html.parser')

    # Create soup from the balance sheet
    html_bs = requests.get(url_stock_bs)
    soup_bs = bs4.BeautifulSoup(html_bs.content, 'html.parser')

    # Creat soup from the key stats
    htm_ks = requests.get(url_stock_ks)
    soup_ks = bs4.BeautifulSoup(htm_ks.content, 'html.parser')

    # Reduced soup as needed
    new_soup_bs = soup_bs.find('div', attrs = {'class':'D(tbrg)'})
    new_soup_header = soup_gen.find('div',attrs = {'id':'quote-header-info'})
    new_soup_body = soup_gen.find('div',attrs = {'id':'quote-summary'})
    new_soup_ks = soup_ks.find('div', attrs = {'class':'Fl(start) W(50%) smartphone_W(100%)'})
   
    # calc from balance sheet
    Asset = str(re.findall('"74">.{1,10},000</', str(new_soup_bs)))[7:-4]
    Liability = str(re.findall('"92">.{1,10},000</', str(new_soup_bs)))[7:-4]

    # Book Value calc
    BVPS_str = str(re.findall('Book Value Per Share.{1,300}', str(new_soup_ks)))
    BVPS = str(re.findall('\d{1,4}\.\d{1,4}', str(BVPS_str)))[2:-2]
    
    # ROE calc
    ROE_str = str(re.findall('Return on Equity.{1,300}\%', str(new_soup_ks)))
    ROE = str(re.findall('\d{1,4}\.\d{1,2}%', str(ROE_str)))[2:-2]    
    
    # calc from header
    Price = str(re.findall('"50">\d?\,?\d{1,3}\.\d{1,4}', str(new_soup_header)))[7:-2]
    Price = Price.replace(',', '')

    # calc from body
    EPS = str(re.findall('"154">\-?\d{1,4}\.\d{1,4}', str(new_soup_body)))[8:-2]
    Beta = str(re.findall('"144">\d{1,4}\.\d{1,4}', str(new_soup_body)))[8:-2]
    Market_cap = str(re.findall('"139">\d{1,4}\.\d{1,4}\w', str(new_soup_body)))[8:-2]
        
    IV = round(math.sqrt(15*1.5*float(EPS)*float(BVPS)),2)
    print('Intensic Value: ' + str(IV))
    print('Stock Price: ' + str(Price))
    print('Diff: ' + str(round(float(IV)-float(Price),2)))
    print('-----')
    print('P/E: ' + str(round(float(Price)/float(EPS),2)))
    print('ROE: ' + str(ROE))
    print('EPS: ' + str(EPS))
    print('Beta: ' + str(Beta))
    print('-----')

In [236]:
stock_value(stock='GM')

Intensic Value: 69.4
Stock Price: 60.30
Diff: 9.1
-----
P/E: 9.73
ROE: 18.33%
EPS: 6.20
Beta: 1.36
-----
